In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\JAGADEESHNAIKPALTHYA\langgraph-end-to-end\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [4]:
llm.invoke("hi,how are you?")

AIMessage(content="As an AI, I don't have feelings, but I'm here and ready to assist you! How can I help you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 15, 'total_tokens': 48, 'completion_time': 0.06, 'prompt_time': 0.001921046, 'queue_time': 0.23514460399999998, 'total_time': 0.061921046}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-76698f2d-32d2-411a-9f56-105dc234a95d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 33, 'total_tokens': 48})

### Pre-difined Tools

In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool.run({"query": "langchan"}))

No good Wikipedia Search Result was found


In [6]:
from langchain_community.tools import YouTubeSearchTool
tool2=YouTubeSearchTool()
tool2.run("sunny savita")

"['https://www.youtube.com/watch?v=cydntJObPLI&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=uQY_7LKS8sc&pp=ygUMc3Vubnkgc2F2aXRh']"

In [7]:
from langchain_community.tools import YouTubeSearchTool
tool2=YouTubeSearchTool()
tool2.run("carry minati")

"['https://www.youtube.com/watch?v=m9s1NQG3TNY&pp=ygUMY2FycnkgbWluYXRp', 'https://www.youtube.com/watch?v=GOFQN8otiYs&pp=ygUMY2FycnkgbWluYXRp']"

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'title': 'Burning Man flooding: What happened to stranded festivalgoers?',
  'url': 'https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331',
  'content': '2:49\nAttendees look at a double rainbow over flooding on a desert plain at the Burning Man festival in Black Rock City, Nevada, September 1, 2023.\nJulie Jammot/AFP via Getty Images\nTens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend. [...] Camps are set on a muddy desert plain at the Burning Man festival in Black Rock, City, Nevada, Sept. 2, 2023.\nJulie Jammot/AFP via Getty Images\nWhat happened?\nBlack Rock Desert was faced with two to three months worth of rain in just a matter of hours on Friday, Sept. 1.\nOn average, the area gets only 0.2 inches of rain or less in September -- but festivalgoers were met with up to 1 inch in some areas, in a desert that gets only about 5 to 6 inches

### Custom tools

In [9]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [10]:
get_word_length.invoke('Hii')

3

In [11]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [15]:
@tool
def summisation(a: int, b: int) -> int:
    """Adding two numbers."""
    return a + b

In [16]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


### Agents

In [17]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [18]:
tool=load_tools(["wikipedia"],llm=llm)

In [19]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\JAGADEESHNAIKPALTHYA\AppData\Local\Temp\ipykernel_22364\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [20]:
agent.run('who is keerthi suresh')

C:\Users\JAGADEESHNAIKPALTHYA\AppData\Local\Temp\ipykernel_22364\895738560.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run('who is keerthi suresh')




> Entering new AgentExecutor chain...
Thought: I need to look up Keerthi Suresh on Wikipedia to find information about her.
Action: wikipedia
Action Input: Keerthi Suresh
Observation: Page: Keerthy Suresh
Summary: Keerthy Suresh (born 17 October 1992) is an Indian actress who appears predominantly in  Tamil and Telugu films. Her accolades include one National Film Award, five SIIMA Awards and two Filmfare Awards South. Keerthy was placed in Forbes India's 30 Under 30 list of 2021. 
Keerthy is the daughter of film producer G. Suresh Kumar and actress Menaka. She began her career as a child actress in the early 2000s and returned to films after studying fashion design. She had her first lead role in the 2013 Malayalam film Geethaanjali, for which she won the SIIMA Award for Best Female Debut – Malayalam. After winning the SIIMA Award for Best Female Debut – Tamil for Idhu Enna Maayam (2015), Keerthy went on to star in successful films such as Ring Master (2014), Nenu Sailaja (2016), Ra

'Keerthi Suresh is an Indian actress who predominantly appears in Tamil and Telugu films.'

In [21]:
agent.run("what is llama and who create this llm model?")



> Entering new AgentExecutor chain...
Thought: I need to search Wikipedia for information about "Llama" to find out what it is and who created it.
Action: wikipedia
Action Input: Llama (large language model)
Observation: Page: Llama (language model)
Summary: Llama (Large Language Model Meta AI, formerly stylized as LLaMA) is a family of large language models (LLMs) released by Meta AI starting in February 2023. The latest version is Llama 3.3, released in December 2024.
Llama models are trained at different parameter sizes, ranging between 1B and 405B. Initially  only a foundation model, starting with Llama 2, Meta AI released instruction fine-tuned versions alongside foundation models.
Model weights for the first version of Llama were only available to researchers on a case-by-case basis, under a non-commercial license. Unauthorized copies of the first model were shared via BitTorrent. Subsequent versions of Llama were made accessible outside academia and released under licenses tha

'Llama is a family of large language models (LLMs) released by Meta AI.'

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [23]:
search = TavilySearchResults()

In [24]:
search.invoke("what is the weather in SF")

[{'title': "Today's Weather in San Francisco Weather Report for 03-04-2025",
  'url': 'https://it-it.spreaker.com/episode/today-s-weather-in-san-francisco-weather-report-for-03-04-2025--64688879',
  'content': "Today's Weather in San Francisco\n Weather Report for 03-04-2025\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDescrizione\n\nGood morning, San Francisco! This is your friendly neighborhood weather forecast, bringing you the scoop on what Mother Nature has in store.\n\nTonight, we're looking at a mostly cloudy evening...\n\nCommenti\n\nInformazioni\n\nAutore | QP-1\nOrganizzazione | William Corbin\nSito | -\nTag | #forecast#rain forecast#weather#weather forecast#weather report\nCopyright 2025 - Spreaker Inc. an\n        iHeartMedia Company [...] Sembra che non tu non abbia alcun episodio attivo\n\nSfoglia il catalogo di Spreaker per scoprire nuovi contenuti\n\nCorrente\n\n\n\n\n\nSembra che non ci sia nessun episodio nella tua coda\n\nSfoglia il catalogo di Spreaker per scoprire nuovi cont

In [25]:
tools = [search]

In [26]:
from langchain import hub

In [27]:
prompt=hub.pull("hwchase17/openai-functions-agent")

In [28]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [29]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [30]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [31]:
agent_executor.invoke({"input": "What is the corrent GDP of India"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'What is the current GDP of India?'}`


[{'title': 'India GDP - Worldometer', 'url': 'https://www.worldometers.info/gdp/india-gdp/', 'content': 'Population\nMore \nWGDPGDP by Country India GDP\n India GDP\nGDP nominal (2022)\n$3,385,090,000,000\n(Current US Dollars). Sources include: World Bank, United Nations.\nGDP growth rate (2022)\n7.00% \nNominal (current) Gross Domestic Product (GDP) of India is $3,385,090,000,000 (USD) as of 2022.\nReal GDP (constant, inflation adjusted) of India reached $2,432,020,000,000 in 2022. [...] GDP Growth Rate in 2022 was 7.00%, representing a change of 193,390,000,000 US$ over 2021, when Real GDP was $2,761,590,000,000.\nGDP per Capita in India (with a population of 1,425,423,212 people) was $2,073 in 2022, an increase of $120 from $1,953 in 2021; this represents a change of 6.2% in GDP per capita.\nCountryContinentWorld\nIndia GDP (Nominal, $USD) 2003-2022

{'input': 'What is the corrent GDP of India', 'output': '\n\n\n\n'}

In [32]:

print(agent_executor.invoke({"input": "whats the weather in new york"}))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in new york'}`


[{'title': 'Weather in New York', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.7142, 'lon': -74.0064, 'tz_id': 'America/New_York', 'localtime_epoch': 1743657971, 'localtime': '2025-04-03 01:26'}, 'current': {'last_updated_epoch': 1743657300, 'last_updated': '2025-04-03 01:15', 'temp_c': 7.2, 'temp_f': 45.0, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 21.5, 'wind_kph': 34.6, 'wind_degree': 124, 'wind_dir': 'SE', 'pressure_mb': 1026.0, 'pressure_in': 30.31, 'precip_mm': 2.0, 'precip_in': 0.08, 'humidity': 76, 'cloud': 100, 'feelslike_c': 2.6, 'feelslike_f': 36.7, 'windchill_c': 3.0, 'windchill_f': 37.5, 'heatindex_c': 6.9, 'heatindex_f': 44.4, 'dewpoint_c': 4.4, 'dewpoint

In [33]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [34]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

In [35]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

In [36]:
vector = FAISS.from_documents(documents,embeddings)
retriever = vector.as_retriever()

In [38]:
retriever.invoke("what is longsmit")[0]

Document(id='da431528-1879-4c21-9471-b3dd222e49cc', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nObservabilityAnalyze traces in LangSmith

In [39]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [40]:
tools = [search, retriever_tool]

In [41]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [42]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [43]:
agent_executor.invoke({"input": "hi! what is a langsmith?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'what is a langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development -

{'input': 'hi! what is a langsmith?',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.'}

In [44]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in san francisco'}`


[{'title': 'Weather in san francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1743658133, 'localtime': '2025-04-02 22:28'}, 'current': {'last_updated_epoch': 1743657300, 'last_updated': '2025-04-02 22:15', 'temp_c': 10.6, 'temp_f': 51.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 11.2, 'wind_kph': 18.0, 'wind_degree': 306, 'wind_dir': 'NW', 'pressure_mb': 1014.0, 'pressure_in': 29.93, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 74, 'cloud': 25, 'feelslike_c': 8.3, 'feelslike_f': 47.0, 'windchill_c': 7.5, 'windchill_f': 45.5, 'heatindex_c': 10.6, 'heatindex_f': 51.0, 'de

{'input': 'whats the weather in sf?',
 'output': 'The weather in San Francisco on April 3rd, 2025 is predicted to be sunny with a high of 59° and a low of 46°.\n'}

### React Agent

In [45]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

In [47]:
google_search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [48]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [49]:
prompt = PromptTemplate.from_template(template)

In [50]:
search_agent = create_react_agent(llm,tools,prompt)

In [51]:
agent_executor = AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [52]:
agent_executor.invoke({"input": "Where is the hometown of the 2007 US PGA championship winner and his score?"})



> Entering new AgentExecutor chain...
Thought: I need to find out who won the 2007 US PGA Championship and where they are from. I also need to know their score. 
Action: Intermediate Answer
Action Input: Who won the 2007 US PGA Championship?The 2007 PGA Championship was the 89th PGA Championship, played August 9–12 at Southern Hills Country Club in Tulsa, Oklahoma. Defending champion Tiger Woods won his fourth PGA Championship and 13th major title, two strokes ahead of runner-up Woody Austin.Thought: Tiger Woods won the 2007 US PGA Championship.  I need to find out where he is from.
Action: Intermediate Answer
Action Input: Where is Tiger Woods from?Cypress, CAThought: Tiger Woods is from Cypress, CA and his score was two strokes ahead of the runner-up. 
Final Answer: Tiger Woods, who is from Cypress, CA, won the 2007 US PGA Championship. 


> Finished chain.


{'input': 'Where is the hometown of the 2007 US PGA championship winner and his score?',
 'output': 'Tiger Woods, who is from Cypress, CA, won the 2007 US PGA Championship.',
 'intermediate_steps': [(AgentAction(tool='Intermediate Answer', tool_input='Who won the 2007 US PGA Championship?', log='Thought: I need to find out who won the 2007 US PGA Championship and where they are from. I also need to know their score. \nAction: Intermediate Answer\nAction Input: Who won the 2007 US PGA Championship?'),
   'The 2007 PGA Championship was the 89th PGA Championship, played August 9–12 at Southern Hills Country Club in Tulsa, Oklahoma. Defending champion Tiger Woods won his fourth PGA Championship and 13th major title, two strokes ahead of runner-up Woody Austin.'),
  (AgentAction(tool='Intermediate Answer', tool_input='Where is Tiger Woods from?', log='Thought: Tiger Woods won the 2007 US PGA Championship.  I need to find out where he is from.\nAction: Intermediate Answer\nAction Input: Wher

### ReAct agent with custom tools

In [53]:
from langchain.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

In [54]:
# Custom tool for the Agent 
@tool
def get_employee_id(name):
  """
  To get employee id, it takes employee name as arguments
  name(str): Name of the employee
  """
  fake_employees = {
    "Alice": "E001",
    "Bob": "E002",
    "Charlie": "E003",
    "Diana": "E004",
    "Evan": "E005",
    "Fiona": "E006",
    "George": "E007",
    "Hannah": "E008",
    "Ian": "E009",
    "Jasmine": "E010"}
  
  return fake_employees.get(name,"Employee not found")

In [55]:
# Custom tool for the Agent 
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")

In [56]:
# Saved React Prompt in langchain hub, we could manually type the prompt as well.
prompt = hub.pull("hwchase17/react")

In [57]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [58]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [59]:
tools = [get_employee_salary, get_employee_id]

In [60]:
agent = create_react_agent(llm,tools,prompt)

In [61]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [62]:
agent_executor.invoke({"input":"What is the Salary of Evan?"})



> Entering new AgentExecutor chain...
Thought: I need to find Evan's employee ID first, then use that to get his salary.
Action: get_employee_id
Action Input: EvanE005Thought: Now that I have Evan's employee ID, I can get his salary. 
Action: get_employee_salary
Action Input: E00545000Thought: I now know the final answer
Final Answer: The salary of Evan is 45000. 


> Finished chain.


{'input': 'What is the Salary of Evan?',
 'output': 'The salary of Evan is 45000.'}

### Self Ask with serach agent

In [63]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.llms import Fireworks
from langchain_community.tools.tavily_search import TavilyAnswer

In [64]:
tools = [TavilyAnswer(max_results=1, name="Intermediate Answer")]

In [65]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")

In [66]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'self-ask-with-search', 'lc_hub_commit_hash': '9ef33d1fc82608bdfaa88b3757306ec4f22511b736cb267b1737616641dac0cc'} template='Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: W

In [67]:
# Construct the Self Ask With Search Agent
agent = create_self_ask_with_search_agent(llm, tools, prompt)

In [68]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [70]:
agent_executor.invoke(
    {"input": "who won more IPL trophies"}
)



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output: Yes, follow-up questions are needed.


Let's break down why and how to proceed:

**Why follow-up questions are needed:**

* **Ambiguity:**  The question "who won more IPL trophies" is too broad.  It needs to specify:
    * **Which teams** are being compared? 
    * **Over what period?**  IPL history spans many years.

**Possible follow-up questions:**

1. **"Which two IPL teams are you comparing?"**  For example: "Who won more IPL trophies, Mumbai Indians or Chennai Super Kings?" 
2. **"Are you asking about IPL wins in the entire history of the tournament, or in a specific year or range of years?"**


Let me know if you'd like to refine the question and I can help you find the answer! 

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPU